# Objektorientierte Programmierung

Ziel: Rechnen mit "Zahlen" auf Restklassenkörpern (Modul-Arithmetik).

Reste bei Addition und Multiplikation modulo n bilden einen Körper, wenn n eine Primzahl ist.

Wir definieren eine Klasse `FiniteFieldElement` (Körper: engl. field), die die Addition und Multiplikation modulo p (prim) implementiert.

In [1]:
class FiniteFieldElement:
    # Konstruktor
    def __init__(self, z, p):
        self.p = p
        self.z = z % p

    def __add__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Addition ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z + z2.z) % self.p, self.p)

    def __mul__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Multiplikation ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z * z2.z) % self.p, self.p)
    
    # Darstellung des Objekts als String
    def __str__(self):
        return f"{self.z} mod {self.p}"

    # Berechne additives Inverses
    def additive_inverse(self):
        return -self.z % self.p

In [2]:
z1 = FiniteFieldElement(4, 7)
print(z1)

4 mod 7


In [3]:
z2 = FiniteFieldElement(5, 7)
print(f"{z1} + {z2} = {z1 + z2}")
print(f"{z1} * {z2} = {z1 * z2}")
print(f"Das additive inverse Element von {z1} ist {z1.additive_inverse()}.")

4 mod 7 + 5 mod 7 = 2 mod 7
4 mod 7 * 5 mod 7 = 6 mod 7
Das additive inverse Element von 4 mod 7 ist 3.


Für die Division brauchen wir einen Exkurs in rekursive Funktionen.

### Rekursive Funktionen
Eine rekursive Funktion ist eine Funktion, die sich in ihrer Definition selbst aufruft.

**Beispiel:** Fakultätsfunktion (engl: factorial).
$$ n! := n \cdot (n-1) \cdot (n-2) \cdot \dots \cdot 1.$$

In [4]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

factorial(100)

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

#### Der euklidische Algorithmus
Bestimmung des größten gemeinsamen Teilers (ggT, engl. greatest common divisor -- gcd) zweier ganzer Zahlen $m$, $n$.

In [5]:
def gcd(m, n):
    if n == 0:
        return m
    return gcd(n, m%n)

gcd(91, 28)

7

#### Der erweiterte euklidische Algorithmus
Math. Fakt: der ggT von $m$ und $n$ ist die kleinste nichtnegative Zahl, die sich als ganzzahlige Linearkombination von $m$ und $n$ schreiben lässt. Wenn $p=ggT(m, n)$, dann gibt es ganze Zahlen $x, y$, so dass
$$ p = x m + y n.$$

Rekursion:
\begin{align}
 ggT(m, n) &= xm + yn\\
      &= ggT(n, m\%n) = x_1 n + y_1 (m\%n) = x_1 n + y_1 \Big(m - (m//n)n\Big)\\
      &= y_1 m + \Big( x_1 - y_1 (m//n) \Big) n
\end{align}

Es ist also
\begin{align}
    x &= y_1 \\
    y &= x_1 - (m//n) y_1
\end{align}

Hierüber können wir den erweiterten euklidischen Algorithmus definieren. Abbruch: bei $n=0$: 
$$ p = x m + y n$$
mit $x=1$ und $y=0$.

In [6]:
def extended_euclid(m, n):
    """
    Liefert den ggT von m und n, sowie zwei ganze Zahlen x und y, so dass ggT = x*m + y*n ist.
    Input: m, n
    Output ggT (gcd), x, y
    """
    if n == 0:
        return (m, 1, 0)
    gcd, x1, y1 = extended_euclid(n, m%n)
    x = y1
    y = x1 - (m//n)*y1
    return (gcd, x, y)

In [7]:
extended_euclid(91, 28)

(7, 1, -3)

#### Berechnung von Inversen in Primzahlkörpern
Die Inverse können wir mit Hilfe des erweiterten euklidischen Algorithmus berechnen.
Für jede Zahl $z$ aus dem Primzahlkörper $F_p$ (Addition und Multiplikation modulo $p$) gilt nämlich:
    $$ \text{ggT}(z, p) = 1.$$
Also gibt es $x,y$, so dass: $$ 1 = xz + yp.$$
Folglich ist $$ xz = 1 \text{ mod } p, $$
d.h., $x$ ist multiplikatives Inverses von $z$.

In [9]:
class FiniteFieldElement:
    # Konstruktor
    def __init__(self, z, p):
        self.p = p
        self.z = z % p

    def __add__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Addition ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z + z2.z) % self.p, self.p)

    def __mul__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Multiplikation ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z * z2.z) % self.p, self.p)
    
    # Darstellung des Objekts als String
    def __str__(self):
        return f"({self.z} mod {self.p})"

    # Darstellung des Objekts für den Entwickler
    def __repr__(self):
        return f"({self.z} mod {self.p})"

    # Berechne additives Inverses
    def additive_inverse(self):
        return -self.z % self.p

    def multiplicate_inverse(self):
        gcd, x, y = extended_euclid(self.z, self.p)
        # return x % self.p
        return FiniteFieldElement(x, self.p)

    # Und dies erlaubt dann auch die Division durch ein Feldelement - dargestellt
    # durch den Operator /, der der Funktion __truediv__ entspricht. (Im Gegensatz
    # zu __floordiv__, die den Operator // implementiert.
    def __truediv__(self, z2):
        return self * z2.multiplicate_inverse()

In [13]:
z65_137 = FiniteFieldElement(65, 137)
print(z65_137.multiplicate_inverse())

(78 mod 137)


In [15]:
65*78 % 137

1

In [16]:
z5_7 = FiniteFieldElement(5, 7)
z5_7.multiplicate_inverse()

(3 mod 7)

In [17]:
z5_7 / z5_7

(1 mod 7)

In [19]:
z2_7 = FiniteFieldElement(2, 7)
z5_7 = FiniteFieldElement(5, 7)
frac = z2_7 / z5_7
f"""{z2_7} dividiert durch {z5_7} ist {frac}, und {frac} * {z5_7} ist {frac*z5_7}."""

'(2 mod 7) dividiert durch (5 mod 7) ist (6 mod 7), und (6 mod 7) * (5 mod 7) ist (2 mod 7).'

### Effiziente ganzzahlige Potenzen
Idee: Iteriertes Quadrieren, z.B.
$$ 3^5 = (3^2)^{(2*1)} * 3^{2*0} * 3^{1}, $$

wobei die Binärdarstellung der $5 = 101_b$ genutzt wurde.

Multiplikation mit 2 im Binärsystem: Verschieben der Bits nach links (rechts Auffüllen mit 0).

In [49]:
1 << 10

1024

Entsprechend ist die ganzzahlige Division durch 2 eine Rechtsverschiebung:

In [50]:
1024 >> 2

256

Weitere bitweise Operationen: Bitweises UND &, bitweises ODER |, bitweises ENTWEDER ODER (XOR) ^.

In [51]:
# Bitweises UND
print(f"{7 & 2 = }")
# Bitweises ODER
print(f"{7 | 2 = }")
# Bitweises ENTWEDER ODER
print(f"{7 ^ 2 = }")

7 & 2 = 2
7 | 2 = 7
7 ^ 2 = 5


In [52]:
# Potenzieren
def pow(base, exponent):
    prod = 1
    b = base
    e = exponent
    while e:
        if e & 1:
            prod *= b
        b *= b
        e = e >> 1
    return prod

In [53]:
pow(2, 13)

8192

Einbau in Klasse `FiniteFieldElement`.

In [65]:
def pow(base, exponent):
    b = base
    e = exponent
    result = 1
    while e:
        if e & 1:
            result *= b
        b *= b
        e >>= 1
    return result

class FiniteFieldElement:
    # Konstruktor
    def __init__(self, z, p):
        self.p = p
        self.z = z % p

    def __add__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Addition ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z + z2.z) % self.p, self.p)

    def __mul__(self, z2):
        if self.p != z2.p:
            raise TypeError("Die Multiplikation ist nicht definiert für Elemente unterschiedlicher Körper")
        return FiniteFieldElement( (self.z * z2.z) % self.p, self.p)
    
    # Darstellung des Objekts als String
    def __str__(self):
        return f"({self.z} mod {self.p})"

    # Darstellung des Objekts für den Entwickler
    def __repr__(self):
        return f"[{self.z} mod {self.p}]"

    # Berechne additives Inverses
    def additive_inverse(self):
        return -self.z % self.p

    def multiplicate_inverse(self):
        gcd, x, y = extended_euclid(self.z, self.p)
        # return x % self.p
        return FiniteFieldElement(x, self.p)

    # Und dies erlaubt dann auch die Division durch ein Feldelement - dargestellt
    # durch den Operator /, der der Funktion __truediv__ entspricht. (Im Gegensatz
    # zu __floordiv__, die den Operator // implementiert.
    def __truediv__(self, z2):
        return self * z2.multiplicate_inverse()

    def __pow__(self, exponent):
        one = FiniteFieldElement(1, self.p)
        b = self * one
        e = exponent
        result = one
        while e:
            if e & 1:
                result *= b
            b *= b
            e >>= 1
        return result

In [63]:
z3_5 = FiniteFieldElement(3, 5)
print(f"{z3_5 ** 130 = }")
print(f"{z3_5 / z3_5 = }")

z3_5 ** 130 = [4 mod 5]
z3_5 / z3_5 = [1 mod 5]


In [66]:
z65_137 = FiniteFieldElement(65, 137)
z13_137 = FiniteFieldElement(13, 137)
(z13_137 / z65_137)

[55 mod 137]

In [69]:
65 * 55 % 137

13